In [1]:
import pipeline_models as pl
import pandas as pd
import numpy as np
import os

from IPython.display import display, HTML
from IPython.display import clear_output

import time

# def set_random(seed_value):
#     # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
#     os.environ['PYTHONHASHSEED']=str(seed_value)
#     # 2. Set `python` built-in pseudo-random generator at a fixed value
#     random.seed(seed_value)
#     # 3. Set `numpy` pseudo-random generator at a fixed value
#     np.random.seed(seed_value)
#     # 4. Set `tensorflow` pseudo-random generator at a fixed value
#     tf.random.set_seed(seed_value)

# set_random(0)

In [2]:
# np.random.seed(0)
# np.random.choice(10, 3, replace=False).tolist()
# np.random.seed(0)
# np.random.get_state()[1][0]

In [3]:
start_time = time.perf_counter()
start_time_ns = time.perf_counter_ns()

In [4]:
stop_time = time.perf_counter()
stop_time_ns = time.perf_counter_ns()

In [5]:
stop_time - start_time

22.067291599999862

In [8]:
(stop_time_ns - start_time_ns)/1000000000

22.0672812

In [6]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./data/"):
    for file in files:
        if file.endswith(".csv"):
            all_files.append(os.path.join(root, file))
            all_files_name.append(root.split("/")[-1] + "_" + file.replace(".", "_"))
# df = pd.read_csv()
for i in range(1,30):
    df = pd.read_csv(all_files[i], 
                sep=';', 
                index_col='datetime', 
                parse_dates=True)

    autoencoder = pl.PipelineModel()
    autoencoder.apply_standard_scaling(())
    autoencoder.apply_nested_rotations((2, 0.75))
    autoencoder.apply_feature_bagging(())
    autoencoder.apply_model(("CONV_AE", "task_name_1"))

    autoencoder.pipeline_fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))

    final_df = autoencoder.pipeline_transform(df.drop(['anomaly', 'changepoint'], axis=1))

    final_df["final_anomaly"] = (final_df["anomaly_by_conv_ae_task_name_1_score"] > (3/2) * autoencoder.model.UCL).astype(int)
    print(f'MAX: {final_df["anomaly_by_conv_ae_task_name_1_score"].max()}, {3/2*autoencoder.model.UCL}, {(3/2)*autoencoder.model.UCL}')
    print(autoencoder.model.model_name)
    print(autoencoder.model.model)
    print(autoencoder.model.UCL)
    print(autoencoder.model.anomaly_col)
    display(final_df.groupby(["final_anomaly"]).count())

MAX: 11.741616997918964, 8.262011242341275, 8.262011242341275
CONV_AE
5.5080074948941835
None


,anomaly_by_conv_ae_task_name_1_score
final_anomaly,
0,684
1,61


MAX: 54.05286290427917, 7.916606743144294, 7.916606743144294
CONV_AE
5.277737828762863
None


,anomaly_by_conv_ae_task_name_1_score
final_anomaly,
0,739
1,588


MAX: 12.765544547924751, 8.92172660441659, 8.92172660441659
CONV_AE
5.9478177362777265
None


,anomaly_by_conv_ae_task_name_1_score
final_anomaly,
0,716
1,474


KeyboardInterrupt: 

In [7]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./data/"):
    for file in files:
        if file.endswith(".csv"):
            all_files.append(os.path.join(root, file))
            all_files_name.append(root.split("/")[-1] + "_" + file.replace(".", "_"))
df = pd.read_csv(all_files[1], 
                sep=';', 
                index_col='datetime', 
                parse_dates=True)
print(df.shape)
autoencoder = pl.PipelineModel()
autoencoder.apply_standard_scaling(())
autoencoder.apply_nested_rotations((2, 0.75))
autoencoder.apply_feature_bagging(())

autoencoder.apply_model(("CONV_AE", "task_name_1"))


autoencoder.pipeline_fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))
a = autoencoder.pipeline_transform(df.drop(['anomaly', 'changepoint'], axis=1))
print(f"{(3/2) * autoencoder.model.UCL} - {a['anomaly_by_conv_ae_task_name_1_score'].max()}")
a["final_anomaly"] = (a['anomaly_by_conv_ae_task_name_1_score'] > (3/2)*autoencoder.model.UCL).astype(int)
print(a.shape)
display(a.groupby(['final_anomaly']).count())
display(a)

(745, 10)
8.226389302690688 - 11.81454586124591
(745, 2)


,anomaly_by_conv_ae_task_name_1_score
final_anomaly,
0,671
1,74


,anomaly_by_conv_ae_task_name_1_score,final_anomaly
datetime,,
2020-03-01 15:44:06,0.0,0
2020-03-01 15:44:07,0.0,0
2020-03-01 15:44:08,0.0,0
2020-03-01 15:44:09,0.0,0
2020-03-01 15:44:10,0.0,0
...,...,...
2020-03-01 15:57:02,0.0,0
2020-03-01 15:57:03,0.0,0
2020-03-01 15:57:04,0.0,0


In [8]:
df.groupby(["anomaly"]).count()

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,changepoint
anomaly,,,,,,,,,
0.0,557,557,557,557,557,557,557,557,557
1.0,188,188,188,188,188,188,188,188,188
